# TD 3: Focus on biases
In this practical class, we will investigate if there are some biases in the data and/or in the model trained on these data, and try to debiase the features. We will focus on the SVM (C = 1, kernel = rbf) classifier you trained in the first practical class on read speech with the eGeMAPS features.\
\
BEFORE DOING ANYTHING ELSE\
**Task n°1. Install the pingouin and statsmodels Python package.**

In [ ]:
!pip install statsmodels
!pip install pingouin

SyntaxError: invalid syntax (2125881801.py, line 1)

# Imports

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import pingouin as pg
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.base import clone


Note: you may need to restart the kernel to use updated packages.Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Elise\anaconda3

  added / updated specs:
    - pingouin


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.10.5  |       h4c7d964_0         153 KB  conda-forge
    certifi-2025.10.5          |     pyhd8ed1ab_0         156 KB  conda-forge
    littleutils-0.2.4          |     pyhd8ed1ab_1          14 KB  conda-forge
    openssl-3.1.8              |       ha4e3fda_0         7.1 MB  conda-forge
    outdated-0.2.2             |     pyhd8ed1ab_2          13 KB  conda-forge
    pandas-flavor-0.7.0        |     pyhd8ed1ab_0          14 KB  conda-forge
    pingouin-0.5.5             |     pyhd8ed1ab_1         162 KB  conda-forge
    ------------------------------------------------

Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (C:\Users\Elise\anaconda3\Lib\site-packages\conda\plugins\types.py))
Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (C:\Users\Elise\anaconda3\Lib\site-packages\conda\plugins\types.py))


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.9.1




# Core Parsing and Analysis Function on the Reading Task

In [ ]:
if __name__ == "__main__":
    # IMPORTANT: Please update this path to your actual data file.
    # The 'r' before the string helps handle Windows backslashes correctly.
    main_file_path = r'/content/combined_data.csv'
    main_directory_column = 'directory'
    
    run_full_analysis(main_file_path, main_directory_column)


def parse_filename(filename):
    """
    Parses a single filename to extract diagnosis, sex, age, and education level.
    Returns a dictionary with all parts or None if no match is found.
    """
    clean_filename = str(filename).strip()
    match = re.match(r'(\d+)_([CP])([FM])(\d{1,2})_(\d+|x)', clean_filename, re.IGNORECASE)

    if match:
        parts = match.groups()
        edu_part = parts[4]
        
        if edu_part.lower() == 'x':
            edu_level = np.nan
        else:
            edu_level = int(edu_part)
            
        return {
            'diagnosis': 'Patient' if parts[1].upper() == 'P' else 'Control',
            'sex': 'Female' if parts[2].upper() == 'F' else 'Male',
            'age': int(parts[3]),
            'education_level': edu_level
        }
    return None

# --- MAIN ANALYSIS FUNCTION ---
def run_full_analysis(file_path, directory_column_name):
    """
    Executes all tasks from 1 to 16.
    """
    # --- DATA PREPARATION (COMMON TO ALL TASKS) ---
    print("--- Preparing Data ---")
    try:
        df_full = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"ERROR: File not found at {file_path}. Please check the path.")
        return

    parsed_df = df_full[directory_column_name].apply(parse_filename).apply(pd.Series)
    df = pd.concat([df_full, parsed_df], axis=1)
    df.dropna(subset=['diagnosis', 'sex', 'age'], inplace=True)
    print(f"Data prepared successfully. Found {len(df)} valid participants.")

Statistical tests in Python\
All the necessary statistical tests can be run directly from the pingouin package, given that your data is already clean and structured on the correct Pandas dataframe.\
Using pingouin:
- χ2 [doc]
- Mann-Whitney’s U [doc]

Using import statsmodels.formula.api as smf:
- Logistic regression: smf.logit("a ∼ b * c * d", data=data).fit()
- Linear regression: smf.ols("a ∼ b * c * d", data=data).fit()

## 1 Biases in data
### 1.1 Univariate bias (*)
**Task n°2. Describe the sex, age, and level of education depending on the diagnosis.**

In [ ]:
 print("\n\n" + "="*50)
    print("SECTION 1: BIASES IN DATA")
    print("="*50)

    # --- Task 2: Describe demographics by diagnosis ---
    print("\n--- Task 2: Descriptive Statistics by Diagnosis ---")
    for group_name, group_df in df.groupby('diagnosis'):
        print(f"\n--- Analysis for '{group_name}' Group ---")
        print("\n-- Age --\n", group_df['age'].describe().loc[['count', 'mean', 'std', 'min', 'max']])
        print("\n-- Sex Distribution --\n", (group_df['sex'].value_counts(normalize=True) * 100).apply(lambda x: f"{x:.2f}%"))
        print("\n-- Education Level Distribution --\n", (group_df['education_level'].value_counts(normalize=True).sort_index() * 100).apply(lambda x: f"{x:.2f}%"))


**Task n°3. Apply statistical tests to identify biases in the data (i.e. differences between the two ‘depressed patient’ and ‘healthy control’ classes). Pick the relevant statistical tests: χ2 for sex, Mann-Whitney’s U for age, χ2 for level of education (cf. box at the end of this file for more details).**

In [ ]:
 print("\n--- Task 3: Statistical Tests for Univariate Bias ---")
    print("\n1. Sex vs. Diagnosis (Chi-squared Test)")
    print(pg.chi2_independence(data=df, x='diagnosis', y='sex')[2])
    
    print("\n2. Age vs. Diagnosis (Mann-Whitney U Test)")
    patient_age = df[df['diagnosis'] == 'Patient']['age']
    control_age = df[df['diagnosis'] == 'Control']['age']
    print(pg.mwu(patient_age, control_age))

    print("\n3. Education Level vs. Diagnosis (Chi-squared Test)")
    print(pg.chi2_independence(data=df, x='diagnosis', y='education_level')[2])

**Q1. Given these results, is the dataset biased?**

Even with the notable difference between Male and Female participant in both groups, the difference isn't significative enough to call the dataset biased

## 1.2 Multivariate bias

In the previous section, we only observed if there were univariate biases (e.g. difference between Male and
Female, independently from their age). In this section, we want to investigate multivariate biases, e.g. if
depression is more prevalent in aged Males with lower education than in young Female with higher education.
On way to do it is to perform a logistic regression, estimating the depression status depending on the
compound effect of the dependant variables.

Task 4

In [ ]:
print("\n--- Task 4: Logistic Regression for Multivariate Data Bias ---")
    df_regr = df.copy()
    df_regr['Dep_status'] = (df_regr['diagnosis'] == 'Patient').astype(int)
    df_regr.dropna(subset=['education_level'], inplace=True)
    formula_data_bias = "Dep_status ~ C(sex) * age * education_level"
    model_data_bias = smf.logit(formula_data_bias, data=df_regr).fit(disp=0)
    print(model_data_bias.summary())

Q2 : What does the equation sex + age + edu.level mean?

This formula represents an additive model. It is used to test the independent effect of each demographic variable (sex, age, education_level) on the diagnosis, assuming the effect of one variable is not influenced by the others. For example, it assumes the effect of age on diagnosis is the same for both males and females.

Q3: This formula represents a multiplicative or interaction model. It is more complex and tests for three things:

The independent ("main") effect of each variable.

The two-way interaction effects (e.g., is the effect of age different for males vs. females?).

The three-way interaction effect (e.g., is the effect of age and education combined different for males vs. females?).

Q4. Interpret the results of the regression. Is there multivariate biases in the dataset?

Based on the logistic regression results from Task 4, the overall model's LLR p-value is  0.7975.

Since this value is greater than 0.05, the model as a whole is not statistically significant. Furthermore, looking at the P>|z| values for the individual interaction terms, none of them are below the 0.05 threshold.

Conclusion: Therefore, we conclude that there is no significant evidence of multivariate bias in the dataset. A combination of demographic factors does not appear to reliably predict the diagnosis.

## 2. Bias in the model

### 2.1 Observing univariate biases

#### 2.1.1 Sex

Task 5 and 6 are incorporated inside task 16

Q5. Is there a link between the performance and the sex of the speaker? What do you conclude?

Looking at the final performance summary table from Task 16, the average UAR for Females (F) is 0.7600 and for Males (M) is 0.6833.

Since these values are different, we can conclude that there is a significant link between the model's performance and the sex of the speaker.

Task 7, 8, 9

In [ ]:
print("\n--- Task 7, 8 & 9: Age Distribution and Categorization ---")
    bins = [0, 30, 40, 50, 60, 101]
    labels = ['<30', '30-39', '40-49', '50-59', '60+']
    df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
    age_group_counts = df['age_group'].value_counts().sort_index()
    print("\nParticipant Count per Merged Age Group:\n", age_group_counts)

    # Plotting age distributions
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.hist(df['age'], bins=15, color='teal', edgecolor='black')
    plt.title('Age Distribution')
    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.subplot(1, 2, 2)
    age_group_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Participants by Age Group')
    plt.xlabel('Age Group')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.suptitle("Age Analysis", fontsize=16)
    plt.show()



Q6. What do you think about the number of participants in each age category?

After merging the extreme categories in Task 9, the participant count for each age group is well-distributed, with each category containing at least 10 individuals. This makes the subsequent performance analysis for each group more statistically reliable and reduces the chance of having no members of a certain class in a test fold.

Q7. Is there a link between performances and age?

Yes, there appears to be a link. The average UAR scores vary across the different age groups, as seen in the final table from Task 16.

The model performs best in the 40-49 group (UAR = 0.8786) and worst in the 30-39 group (UAR = 0.6966). This suggests that the model's effectiveness is influenced by the speaker's age.

Task 10 and 11 covered in Task 16 as well

Q8. Is there a link between the performances of the classifier and the education level of the speakers?

Based on the final table from Task 16, the average UAR scores across the different education levels are: Edu 1: 0.9000, Edu 2 : 0.7786, Edu 3: 0.8024, Edu 4: 0.5182

Since these values are vary significantly, it suggests that there is a link between model performance and the education level of the speakers.

## 2.2 Multivariate biases

Task 12 and 13

In [ ]:
 print("\n--- Task 12 & 13: Logistic Regression for Multivariate Model Bias ---")
    y_preds_standard = cross_val_predict(base_model, X, y_encoded, cv=skf)
    df['correctly_classified'] = (y_encoded == y_preds_standard).astype(int)
    df_regr_model = df.dropna(subset=['education_level'])
    formula_model_bias = "correctly_classified ~ C(sex) * age * education_level"
    model_model_bias = smf.logit(formula_model_bias, data=df_regr_model).fit(disp=0)
    print(model_model_bias.summary())

Q9. Interpret the results of the regression. Is there multivariate biases in the model?

Based on the logistic regression results from Task 13, the overall model's LLR p-value is 0.9290.

Since this is greater than 0.05, the model as a whole is not statistically significant. This indicates that there is no significant evidence that a combination of sex, age, and education level can predict whether the model will classify a speaker correctly. Therefore, we conclude there is no evidence of multivariate bias in the model's performance.

# 3. Debiasing

Task 14 and 15

In [ ]:
 print("\n--- Task 14 & 15: Generating Predictions with Iterative Stratification ---")
    stratify_df = pd.DataFrame({
        'diagnosis': y_encoded,
        'sex': df['sex'].astype('category').cat.codes,
        'age_group': df['age_group'].astype('category').cat.codes
    })
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_preds_iterstrat = np.zeros_like(y_encoded)

    for train_index, test_index in mskf.split(X, stratify_df):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y_encoded[train_index]
        
        fold_model = clone(base_model)
        fold_model.fit(X_train, y_train)
        y_preds_iterstrat[test_index] = fold_model.predict(X_test)
    print("Predictions generated successfully.")


Task 16

In [ ]:
print("\n--- Task 16: Performance Summary using Iterative Stratification ---")
    results = []
    y_true = y_encoded
    y_pred = y_preds_iterstrat

    # Overall
    results.append({'Category': 'Overall', 'UAR': balanced_accuracy_score(y_true, y_pred), 'N': len(df)})
    
    # By Sex
    for sex_group in ['Female', 'Male']:
        mask = (df['sex'] == sex_group)
        results.append({'Category': sex_group[0], 'UAR': balanced_accuracy_score(y_true[mask], y_pred[mask]), 'N': mask.sum()})

    # By Age Group
    for age_group in labels:
        mask = (df['age_group'] == age_group)
        results.append({'Category': age_group, 'UAR': balanced_accuracy_score(y_true[mask], y_pred[mask]), 'N': mask.sum()})

    # By Education Level
    df_edu_temp = df.dropna(subset=['education_level'])
    edu_levels = sorted(df_edu_temp['education_level'].unique())
    for edu_level in edu_levels:
         mask = (df['education_level'] == edu_level)
         results.append({'Category': f'Edu {int(edu_level)}', 'UAR': balanced_accuracy_score(y_true[mask], y_pred[mask]), 'N': mask.sum()})

    results_df_final = pd.DataFrame(results).set_index('Category')
    print(results_df_final.round(4).to_string())
    # Q10 is an interpretation of this final table.